## Laboratorio 5 - Autocodificadores

In [295]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Model

# 1.1 Cargar los datos
df = pd.read_csv('UK_foods.csv', index_col=0)  # Suponiendo que la primera columna contiene los nombres de los alimentos

# Transponer el DataFrame para que los países sean las observaciones y los alimentos las características
df = df.transpose()

print(df.head())

           Cheese  Carcass_meat   Other_meat   Fish  Fats_and_oils   Sugars  \
England       105            245          685   147             193     156   
Wales         103            227          803   160             235     175   
Scotland      103            242          750   122             184     147   
N.Ireland      66            267          586    93             209     139   

           Fresh_potatoes   Fresh_Veg   Other_Veg   Processed_potatoes   \
England                720         253         488                  198   
Wales                  874         265         570                  203   
Scotland               566         171         418                  220   
N.Ireland             1033         143         355                  187   

           Processed_Veg   Fresh_fruit   Cereals   Beverages  Soft_drinks   \
England               360          1102      1472         57          1374   
Wales                 365          1137      1582         73          12

In [296]:
# 1.2 Exploración básica
print(df.describe())

           Cheese  Carcass_meat   Other_meat         Fish  Fats_and_oils   \
count    4.000000           4.00     4.000000    4.000000        4.000000   
mean    94.250000         245.25   706.000000  130.500000      205.250000   
std     18.856917          16.50    93.427334   29.557853       22.366269   
min     66.000000         227.00   586.000000   93.000000      184.000000   
25%     93.750000         238.25   660.250000  114.750000      190.750000   
50%    103.000000         243.50   717.500000  134.500000      201.000000   
75%    103.500000         250.50   763.250000  150.250000      215.500000   
max    105.000000         267.00   803.000000  160.000000      235.000000   

          Sugars  Fresh_potatoes   Fresh_Veg   Other_Veg   \
count    4.00000         4.000000    4.000000    4.000000   
mean   154.25000       798.250000  208.000000  457.750000   
std     15.47848       200.755863   60.188592   92.471167   
min    139.00000       566.000000  143.000000  355.000000   
2

In [297]:
# 1.3 Normalización
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)
print(df_normalized)

             Cheese  Carcass_meat   Other_meat       Fish  Fats_and_oils   \
England    0.658275      -0.017495    -0.259546  0.644585       -0.632429   
Wales      0.535805      -1.277169     1.198856  1.152440        1.535899   
Scotland   0.535805      -0.227441     0.543811 -0.332059       -1.097071   
N.Ireland -1.729885       1.522105    -1.483121 -1.464967        0.193601   

             Sugars  Fresh_potatoes   Fresh_Veg   Other_Veg   \
England    0.130551        -0.450076    0.863312    0.377736   
Wales      1.547958         0.435696    1.093528    1.401682   
Scotland  -0.540853        -1.335847   -0.709834   -0.496364   
N.Ireland -1.137656         1.350227   -1.247006   -1.283054   

           Processed_potatoes   Processed_Veg   Fresh_fruit   Cereals   \
England              -0.336265        0.805477      0.737407 -0.644322   
Wales                 0.084066        1.171603      0.929297  1.679463   
Scotland              1.513193       -0.878702     -0.057567 -0.855575 

In [298]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import numpy as np

# Setear una semilla para evitar mucha variacion entre los datos 
np.random.seed(0)
tf.random.set_seed(0)

# Suponiendo que df_normalized ya existe
# 2.1 Usar todas las observaciones para entrenar
X_train = df_normalized.values

# Número de características o dimensiones
num_features = X_train.shape[1]

# Asegurarse de que el número de características es 17
assert num_features == 17, f"Expected 17 features, but got {num_features}"

# 2.2 Definir la arquitectura
input_layer = layers.Input(shape=(num_features,))
encoder_1 = layers.Dense(12, activation='relu')(input_layer)
encoder_2 = layers.Dense(8, activation='relu')(encoder_1)
encoder_3 = layers.Dense(4, activation='relu')(encoder_2)
bottleneck = layers.Dense(2, activation='relu')(encoder_3)
decoder_1 = layers.Dense(4, activation='relu')(bottleneck)
decoder_2 = layers.Dense(8, activation='relu')(decoder_1)
decoder_3 = layers.Dense(12, activation='relu')(decoder_2)
output_layer = layers.Dense(num_features, activation='sigmoid')(decoder_3)

autoencoder = Model(inputs=input_layer, outputs=output_layer)

# Modelo para la reducción de dimensionalidad
encoder_model = Model(inputs=input_layer, outputs=bottleneck)

# 2.3 Compilación y entrenamiento
autoencoder.compile(loss="mse", optimizer=Adam(lr=0.001))
autoencoder.fit(X_train, X_train, epochs=100, batch_size=1)

# 2.4 Evaluación
loss = autoencoder.evaluate(X_train, X_train)
print(f"Loss: {loss}")


Epoch 1/100
4/4 [==============================] - 1s 5ms/step - loss: 1.2501
Epoch 2/100
4/4 [==============================] - 0s 5ms/step - loss: 1.2495
Epoch 3/100
4/4 [==============================] - 0s 0s/step - loss: 1.2489
Epoch 4/100
4/4 [==============================] - 0s 0s/step - loss: 1.2486
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 1.2480
Epoch 6/100
4/4 [==============================] - 0s 0s/step - loss: 1.2476
Epoch 7/100
4/4 [==============================] - 0s 0s/step - loss: 1.2472
Epoch 8/100
4/4 [==============================] - 0s 5ms/step - loss: 1.2467
Epoch 9/100
4/4 [==============================] - 0s 5ms/step - loss: 1.2463
Epoch 10/100
4/4 [==============================] - 0s 0s/step - loss: 1.2460
Epoch 11/100
4/4 [==============================] - 0s 0s/step - loss: 1.2454
Epoch 12/100
4/4 [==============================] - 0s 5ms/step - loss: 1.2451
Epoch 13/100
4/4 [==============================] - 0s 0s/step - lo

In [299]:
# 3.1 Use the autoencoder to reduce the dimensions of the dataset to 2.
encoded_data = encoder_model.predict(df_normalized)

print(encoded_data.shape)  # Debería mostrar (17, 2) si todo fue correcto

1/1 [==============================] - 0s 53ms/step
(4, 2)


In [300]:


# Asegurando que los datos codificados tienen la forma correcta
assert encoded_data.shape == (4, 2), "La forma de los datos codificados no es correcta."

# Creando un dataframe para las dimensiones codificadas
df_encoded = pd.DataFrame(encoded_data, columns=['Dimension 1', 'Dimension 2'])
df_encoded['Country'] = df_normalized.index  # Usar las filas del DataFrame normalizado como 'Country'

# Dibujo del gráfico
fig = px.scatter(df_encoded, 
                 x="Dimension 1", 
                 y="Dimension 2", 
                 color="Country",
                 title="Representación 2D de los datos codificados por país",
                 width=800,
                 height=600)

fig.show()


## Interpretación de Patrones de Consumo:

### Inglaterra, Escocia y Gales:
Demografía: Estas regiones tienen poblaciones más urbanizadas en comparación con Irlanda del Norte. Las áreas urbanizadas tienden a tener un mayor acceso a una variedad de alimentos, incluidos los alimentos importados y procesados.

Economía: Inglaterra, en particular, tiene una economía más diversificada y un PIB per cápita más alto en comparación con las otras regiones, lo que puede influir en la capacidad de gasto y, por lo tanto, en los patrones de consumo de alimentos.

Cultura: La cultura alimentaria en estas regiones puede ser similar debido a su proximidad geográfica y a la influencia histórica compartida.

### Irlanda del Norte (N.Ireland):
Historia y Política: Irlanda del Norte ha tenido una historia política y social única en comparación con el resto del Reino Unido, con períodos de conflicto y tensiones. Estas circunstancias pueden haber influido en los patrones de comercio, la economía y, por ende, en el consumo de alimentos.

Demografía: Irlanda del Norte es más rural en comparación con las otras regiones. Las áreas rurales pueden depender más de la agricultura local y tener diferentes patrones de consumo.

Economía: La economía de Irlanda del Norte ha sido históricamente más dependiente de la agricultura, lo que podría influir en un consumo más alto de ciertos alimentos frescos o locales.

Cultura: Las tradiciones culinarias de Irlanda del Norte podrían diferir de las del resto del Reino Unido debido a las influencias culturales tanto de la República de Irlanda como del Reino Unido.


#### Con relacion a la grafica obtenida:
Inglaterra, Escocia y Gales:
Estas tres regiones, al estar agrupadas cerca una de la otra en el eje Dimension 2, comparten patrones similares de consumo de alimentos. La urbanización en Inglaterra y Gales podría llevar a patrones de consumo similares, con una variedad más amplia de alimentos disponibles y posiblemente un mayor consumo de alimentos procesados o importados.
La economía diversificada y el mayor PIB per cápita en Inglaterra podrían influir en una dieta más variada, pero, dado que Escocia y Gales están cercanas en la representación, sus patrones de consumo no difieren drásticamente de los de Inglaterra.
Las similitudes culturales y la proximidad geográfica también podrían contribuir a hábitos alimenticios compartidos entre estas regiones.
Irlanda del Norte (N.Ireland):

La notable separación de Irlanda del Norte en el eje Dimension 2 indica un patrón de consumo de alimentos muy diferente en comparación con las otras tres regiones.
La historia política y social única de Irlanda del Norte podría haber llevado a diferentes patrones comerciales y de acceso a ciertos alimentos.
La naturaleza más rural de Irlanda del Norte podría reflejarse en un consumo más alto de alimentos agrícolas frescos y locales. Esto podría explicar por qué se destaca en comparación con las otras regiones que tienen un carácter más urbanizado.
Las tradiciones culinarias distintas en Irlanda del Norte, influenciadas tanto por la República de Irlanda como por el Reino Unido, podrían haber llevado a una preferencia por ciertos alimentos que no son tan populares en las otras regiones.

## Evaluación de Rendimiento del Modelo:

#### Pérdida del Autocodificador:
La pérdida final del autocodificador después de 100 épocas es 1.2090423107147217. Es generalmente aceptado que una pérdida más baja indica una mejor capacidad del modelo para comprimir y reconstruir los datos. Siempre es útil monitorear cómo evoluciona esta pérdida a lo largo de las épocas para determinar si el modelo ha convergido o si podría beneficiarse de más épocas de entrenamiento.

#### Tamaño del Bottleneck:
Se eligio una dimensionalidad de 2 para el bottleneck, lo cual permite visualizar fácilmente los datos en un espacio 2D y es valioso para la interpretación. Sin embargo, es posible que una dimensionalidad mayor capture más variabilidad de los datos y reduzca la pérdida. En el futuro, se considerará experimentar con diferentes tamaños de bottleneck para ver cómo afecta al rendimiento y a la interpretación.

#### Sensibilidad a Hiperparámetros:
Se experimento con diferentes tasas de aprendizaje y se encontró que 0.001 funciona mejor. Esto indica que el modelo es sensible a la tasa de aprendizaje y que elegir el hiperparámetro adecuado puede mejorar significativamente el rendimiento. En futuras investigaciones, también se considerará experimentar con diferentes funciones de activación, arquitecturas de red y otros hiperparámetros.